# Rappel du jour 2 

* tests
    * pyramide de tests
* manipulations de types / structures de données
    * utilisation de `fstrings`
    * dictionnaires et sets 
* début de programmation fonctionnelle
    * les fonctions sont des objets comme les autres 

In [8]:
def toto():
    global tata 
    tata += 1 
    print(id(tata), tata)

In [9]:
tata = 3.234432
print(id(tata), tata)
toto()
print(id(tata), tata)

4429620304 3.234432
4429617904 4.234432
4429617904 4.234432


In [15]:
fs = []
for i in range(5):
    def f(x, nombre=i):
        return x + nombre
    fs.append(f)

print(i)
for f in fs:
    print(f(10))

4
10
11
12
13
14


In [25]:
def spy_list():
    print("dans spy list")
    return []

print("entre spy et append wrong")

def append_wrong(val, li=spy_list()):
    print("dans execution de append wrong")
    li.append(val)
    return li

print("apres append wrong")


entre spy et append wrong
dans spy list
apres append wrong


In [32]:
append_wrong(21)

dans execution de append wrong


[20, 20, 20, 20, 20, 20, 21]

In [44]:
def append_ok(val, li=None):
    if li is None:
        li = []
    print("dans execution de append wrong")
    li.append(val)
    return li

In [47]:
append_ok(3, [1, 2])

dans execution de append wrong


[1, 2, 3]